In [2]:
from sage.all import *
import networkx as nx
import numpy as np
import os
import random
import scipy.stats
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import sys
import time

In [8]:
# target number of graph in the dataset (~300k)
target = 300000
# bucket size for decimal invariants
float_value_range = 0.5
# every graph having score < threshold is discarded
threshold = 25.5

test_limit = 2000

In [3]:
def diameter(G):
    d = Graph(G).diameter()
    return d if not math.isinf(d) else -1

def spectral_radius_lm(G):
    real_value = max(abs(np.linalg.eigvals(nx.laplacian_matrix(G).toarray())))
    return round(real_value / float_value_range) * float_value_range

def spectral_radius_am(G):
    real_value = max(abs(np.linalg.eigvals(nx.adjacency_matrix(G).toarray())))
    return round(real_value / float_value_range) * float_value_range

funcs = {
    'nodes_count': lambda G : len(Graph(G)),
    'diameter': diameter,
    'matching_number': lambda G : len(Graph(G).matching()),
    'clique_number': lambda G : Graph(G).clique_number(),
    'independence_number': lambda G : len(Graph(G).independent_set()),
    'spectral_radius_laplacian': spectral_radius_lm,
    'spectral_radius_adjacency': spectral_radius_am
}

invariant_occurrences = { key: {} for key in funcs.keys() }

graph_set = set()

In [ ]:
#  Valid graph ratio at iteration 200.000: 10%  
def Gnp():
    p = scipy.stats.beta.rvs(0.4, 0.4)  # Beta distribution with higher probability at the extremes
    return nx.fast_gnp_random_graph(n=random.randint(9, 50), p=p)

#  Valid graph ratio at iteration 200.000: 40%  
def GnpLow():
    p = scipy.stats.beta.rvs(2, 5)
    return nx.fast_gnp_random_graph(n=random.randint(9, 50), p=p)

#  Valid graph ratio at iteration 200.000: 40%  
def GnpHigh():
    p = scipy.stats.beta.rvs(5, 2)
    return nx.fast_gnp_random_graph(n=random.randint(9, 50), p=p)

#  Valid graph ratio at iteration 200.000: 30%  
def BarabasiAlbert():
    nodes = random.randint(9, 50)
    return nx.barabasi_albert_graph(n=nodes, m=random.randint(1, nodes - 1))

#  Valid graph ratio at iteration 200.000: 17%  
def RandomTree():
    return nx.random_tree(n=random.randint(9, 50))

#  Valid graph ratio at iteration 200.000: 35%  
def Watts():
    n = random.randint(9, 50)  
    k = random.randint(2, min(n-1, 10))
    p = random.uniform(0.05, 1)
    return nx.watts_strogatz_graph(n, k, p)

def Caveman():
    return  nx.caveman_graph(l=random.randint(2, 10), k=random.randint(2, 10))

def Cycle():
    return nx.cycle_graph(n=random.randint(9, 50))

def Grid():
    m = random.randint(2, 10)
    n = random.randint(2, int(50/m))
    return nx.grid_2d_graph(m, n)

#  Valid graph ratio at iteration 200.000: 45%  
def StochasticBlock():
    total_nodes = random.randint(9, 50)
    num_groups = random.randint(2, 9)
    
    # Random division of nodes in groups
    alpha = np.ones(num_groups)     # vector of ones, with length equal to num_groups
    p = np.random.dirichlet(alpha)  # probability vector from the Dirichlet distribution
    sizes = np.random.multinomial(total_nodes, p)

    # Probs matrix, low dentro groups, high tra groups
    probs = np.full((num_groups, num_groups), random.uniform(0.05, 0.3))  
    np.fill_diagonal(probs, [random.uniform(0.5, 0.9) for _ in range(num_groups)])  

    return nx.stochastic_block_model(sizes, probs)

#  Valid graph ratio at iteration 200.000: 6%  
def Waxman():
    p_beta = scipy.stats.beta.rvs(2, 5)  # higher probability 0-0.3
    p_alpha = scipy.stats.beta.rvs(4, 8) #higher probability 0.2-0.4 
    return nx.waxman_graph(n=random.randint(9, 50), beta=p_beta, alpha=p_alpha)

In [4]:
def graph_hash_function(self):
    hash_str = nx.weisfeiler_lehman_graph_hash(G=self)
    return hash(hash_str)

nx.Graph.__hash__ = graph_hash_function

def save_checkpoint(graph_list, invariant_occurrences, global_scores, accepted_mask, checkpoint_dir, index, hash_set = None):
    with open(os.path.join(checkpoint_dir, 'graph-set', f'graph_set_{index}.pkl'), 'wb') as f:
        pickle.dump(graph_list, file=f)
    with open(os.path.join(checkpoint_dir, 'invariant_occurrences.pkl'), 'wb') as f:
        pickle.dump(invariant_occurrences, file=f)
    with open(os.path.join(checkpoint_dir, 'global_scores.pkl'), 'wb') as f:
        pickle.dump(global_scores, file=f)
    with open(os.path.join(checkpoint_dir, 'accepted_mask.pkl'), 'wb') as f:
        pickle.dump(accepted_mask, file=f)
    if hash_set is not None:
        with open(os.path.join(checkpoint_dir, 'hash_set.pkl'), 'wb') as f:
            pickle.dump(hash_set, file=f)


def restore_checkpoint(checkpoint_dir):
    restored_list = []
    invaraint_occurrences = {}
    global_scores = []
    accepted_mask = []
    hash_set = set()

    with open(os.path.join(checkpoint_dir, 'invariant_occurrences.pkl'), 'rb') as f:
        invaraint_occurrences = pickle.load(file=f)

    with open(os.path.join(checkpoint_dir, 'global_scores.pkl'), 'rb') as f:
        global_scores = pickle.load(file=f)

    with open(os.path.join(checkpoint_dir, 'accepted_mask.pkl'), 'rb') as f:
        accepted_mask = pickle.load(file=f)
    
    with open(os.path.join(checkpoint_dir, 'hash_set.pkl'), 'rb') as f:
        hash_set = pickle.load(file=f)
    
    graph_set_dir_path = os.path.join(checkpoint_dir, 'graph-set')
    for file_name in os.listdir(graph_set_dir_path):
        with open(os.path.join(graph_set_dir_path, file_name), 'rb') as f:
            partial_list = pickle.load(file=f)
            restored_list.extend(partial_list)
    
    return restored_list, invaraint_occurrences, global_scores, accepted_mask, hash_set


def calc_invariants(graph):
    return { key: funcs[key](graph) for key in funcs.keys() }


def calc_invariant_score(invariant, value, total_count, invaraint_occ):
    occurrences = invaraint_occ[invariant].get(value, sys.float_info.epsilon)
    invariant_freq =  occurrences / (total_count or 1)
    invariant_score = -math.log(invariant_freq)
    return invariant_score


def calc_score(graph, total_cont, invariant_occ):
    values = calc_invariants(graph)
    score = 0
    partial_scores = {}
    for invariant in funcs.keys():
        invariant_value = values[invariant]
        partial_scores[invariant] = calc_invariant_score(invariant=invariant, value=invariant_value, total_count=total_cont, invaraint_occ=invariant_occ)
        score += partial_scores[invariant]
    return score, partial_scores, values


def add_graph(graph, invariant_values):
    if graph in graph_set:
        return
    graph_set.add(graph)
    for inv in funcs.keys():
        value = invariant_values[inv]
        if value in invariant_occurrences[inv]:
            invariant_occurrences[inv][value] += 1
        else:
            invariant_occurrences[inv][value] = 1

In [12]:
dir_path = "../plots"

def movingaverage(values, window_size):
    window = np.ones(int(window_size))/float(window_size)
    avg = np.convolve(values, window, 'same')
    return avg


def plot_partial_scores(scorelist, filename, title, sections=None):
    fig, axs = plt.subplots(len(list(scorelist.keys())), 1, figsize=(9, 14), layout="constrained", sharex=True)
    plt.xlabel(f"Iteration")
    plt.ylabel("Score value")
    plt.suptitle(title)
    for i, key in enumerate(list(scorelist.keys())):
        values = np.array(scorelist[key])
        x = np.arange(len(scorelist[key]))
        y = values
        
        if sections is not None:
            x_sections = np.split(x, sections)
            y_sections = np.split(y, sections)
            x_y_sections = zip(x_sections, y_sections)
            for x_y_section in x_y_sections:
                axs[i].plot(x_y_section[0], x_y_section[1], label=key, linewidth=0.6)
                axs[i].set_ylabel(f"{key} score", fontsize=6)
        else:
            axs[i].plot(x, y, label=key, linewidth=0.6)
            axs[i].set_ylabel(f"{key} score", fontsize=6)
        
    plt.savefig(os.path.join(dir_path, filename))
    plt.close()


def plot_general_score(scorelist, filename, validcount, avgwindow, title, sections=None, sectionnames=None):
    values = np.array(scorelist['general'])
    x = np.arange(len(values))
    y = values
    y_avg = movingaverage(y, avgwindow)
    avg_padding = math.ceil(avgwindow/2)
    plt.figure(figsize=(8, 6))
    plt.xlabel(f"Iteration")
    plt.ylabel("Score")
    
    if sections is not None:
        x_sections = np.split(x, sections)
        y_sections = np.split(y, sections)
        x_y_sections = zip(x_sections, y_sections)
        for j, x_y_section in enumerate(x_y_sections):
            plt.plot(x_y_section[0], x_y_section[1], label=sectionnames[j] or j, linewidth=0.6)
    else:
        plt.plot(x, y, label='score', linewidth=0.6)

    # plt.hlines(y=threshold, xmin=0, xmax=test_limit, linewidth=1, color='r', linestyle=':')
    plt.scatter(x[y > y_avg], y[y > y_avg] + 4, color='red', s=10, marker='v', label=f'valid graphs: {validcount}')
    plt.plot(x[avg_padding:-avg_padding], y_avg[avg_padding:-avg_padding], label='moving average', linewidth=1.2)
    # plt.ylim((0, 30)) 

    plt.legend(loc='upper right')
    plt.title(label=title)
    plt.savefig(os.path.join(dir_path, filename))
    plt.close()

In [ ]:
generators = [
    #Gnp,
    GnpLow,     
    GnpHigh,
    BarabasiAlbert,
    RandomTree,
    Watts,
    #Caveman,   #  Valid graph ratio at iteration 200.000: 0%  
    #Cycle,     #  Valid graph ratio at iteration 200.000: 0%  
    #Grid,      #  Valid graph ratio at iteration 200.000: 0%  
    StochasticBlock,
    Waxman
]

window_size = 100

def min_score(values, w_size):
    padded_values = np.pad(values, (w_size, 0), 'constant', constant_values=(0, 0))
    window = padded_values[-w_size:]
    avg = np.mean(window)
    return min(avg, 50)

In [ ]:
for generator in generators:
    graph_set.clear()
    invariant_occurrences = { key: {} for key in funcs.keys() }
    score_list = { key: [] for key in (list(funcs.keys()) + ['general']) }
    i = 0

    while (len(graph_set) < target) and (i < test_limit):
        g = generator()
        score, partial_scores, values = calc_score(g, total_cont=len(graph_set), invariant_occ=invariant_occurrences)

        if score >= min_score(score_list['general'], w_size=window_size):
            add_graph(g, invariant_values=values)
            
        for inv in funcs.keys():
            score_list[inv].append(partial_scores[inv])
        score_list['general'].append(score)
        i += 1

    plot_partial_scores(scorelist=score_list, filename=f"partial_score_{generator.__name__}.pdf", title=generator.__name__)
    plot_general_score(scorelist=score_list, filename=f"general_score_{generator.__name__}.pdf", validcount=len(graph_set), avgwindow=window_size, title=generator.__name__)
    

In [ ]:
graph_set.clear()
invariant_occurrences = { key: {} for key in funcs.keys() }
score_list = { key: [] for key in (list(funcs.keys()) + ['general']) }
i = 0

section_limit = math.floor(test_limit/len(generators))

for generator in generators:
    i = 0
    while (len(graph_set) < target) and (i < section_limit):
        g = generator()
        score, partial_scores, values = calc_score(g, total_cont=len(graph_set), invariant_occ=invariant_occurrences)

        if score >= min_score(score_list['general'], w_size=window_size):
            add_graph(g, invariant_values=values)
        
        for inv in funcs.keys():
            score_list[inv].append(partial_scores[inv])
        score_list['general'].append(score)
        i += 1

plot_partial_scores(scorelist=score_list, filename=f"partial_score_generators.pdf", title="Generators combined", sections=[section_limit * i for i in range(1, len(generators))])
plot_general_score(scorelist=score_list, filename=f"general_score_generators.pdf", validcount=len(graph_set), avgwindow=window_size, title="Generators combined", sections=[section_limit * i for i in range(1, len(generators))], sectionnames=[gen.__name__ for gen in generators])
    

In [ ]:
graph_list = []
hash_set = set()
invariant_occurrences = {key: {} for key in funcs.keys()}
score_list = {key: [] for key in (list(funcs.keys()) + ['general'])}

global_scores = []
accepted_mask = []
already_existing_count = 0

i = 0
checkpoint_index = 0
generator_index = 0
generator = generators[0]

def add_graph_list(graph, invariant_values) -> bool:
    hash_str = nx.weisfeiler_lehman_graph_hash(G=graph)
    if hash_str in hash_set:
        return false
    graph_list.append(graph)
    hash_set.add(hash_str)
    for inv in funcs.keys():
        value = invariant_values[inv]
        if value in invariant_occurrences[inv]:
            invariant_occurrences[inv][value] += 1
        else:
            invariant_occurrences[inv][value] = 1
    return true

start = time.time()
old_valid_count = 0

with open('out.txt', 'w+') as out_file:
    while len(hash_set) < target:
        g = generator()

        score, partial_scores, values = calc_score(g, total_cont=len(hash_set), invariant_occ=invariant_occurrences)

        global_scores.append(score)
        accepted = false

        if score >= min_score(score_list['general'], w_size=window_size):
            accepted = add_graph_list(g, invariant_values=values)
            if not accepted:
                already_existing_count += 1
            
        accepted_mask.append(1 if accepted else 0)

        score_list['general'].append(score)

        i += 1

        if i % 1000 == 0:
            new_valid_count = len(hash_set)
            elapsed_time = time.time() - start
            generator_index = (generator_index + 1) % len(generators)
            
            log_message = (
                f"Previous generator: {generator.__name__}\n"
                f"Iteration: {i}\n"
                f"Elapsed time: {elapsed_time} seconds\n"
                f"Graphs added: {new_valid_count - old_valid_count}\n"
                f"Valid graph ratio: {round(((new_valid_count - old_valid_count)/10),2)}%\n"
                f"Rejected graphs (already existing): {already_existing_count}\n"
                f"Dataset size: {len(hash_set)}\n\n"
            )

            out_file.write(log_message)
            out_file.flush()  # to write in real time

            generator = generators[generator_index]
            old_valid_count = new_valid_count
            already_existing_count = 0
            print(i)
            start = time.time()

        if i % 10000 == 0:
            save_checkpoint(
                graph_list=graph_list, 
                invariant_occurrences=invariant_occurrences, 
                global_scores=global_scores,
                accepted_mask=accepted_mask,
                checkpoint_dir='../data/checkpoints', 
                index=checkpoint_index,
                hash_set=hash_set
            )
            checkpoint_index += 1
            graph_list = []
    
    save_checkpoint(
        graph_list=graph_list, 
        invariant_occurrences=invariant_occurrences, 
        global_scores=global_scores,
        accepted_mask=accepted_mask,
        checkpoint_dir='../data/checkpoints', 
        index=checkpoint_index
    )
        

In [ ]:
graph_list, invariant_occurrences, global_scores, accepted_mask, hash_set = restore_checkpoint(checkpoint_dir="../data/checkpoints")
print(len(graph_list))
print(invariant_occurrences)
print(len(global_scores))
print(len(accepted_mask))
print(np.array(accepted_mask).sum())
print(len(hash_set))
print(np.where(np.array(global_scores) < 0)[0])


In [ ]:
def append_graphs_to_g6(graphs, filepath):
    with open(filepath, "a") as f:
        for i, G in enumerate(graphs):
            g6_str = nx.to_graph6_bytes(G, header=False).decode("utf-8").strip()
            f.write(g6_str + "\n")
            print(i)
            

def write_database_to_file(folder_path, out_path):
    with open(out_path, "w") as outfile:
        for i in range(2, 9):
            with open(os.path.join(folder_path, f"graph{i}.g6"), "r") as database_file:
                outfile.write(database_file.read())    
            outfile.write('\n')

write_database_to_file(folder_path="../../data/database", out_path="../data/raw/dataset.g6")
append_graphs_to_g6(graphs=graph_list, filepath="../data/raw/dataset.g6")

In [5]:
import json

import os
import networkx as nx
import time

def reset_time_with_message(message, t):
    print(f"{message} : {time.time() - t} s")
    return time.time()

def parse_g6_directory(directory: str):
    graphs = []
    if not os.path.isdir(directory):
        raise FileNotFoundError(f"Directory '{directory}' not found.")
    for filename in os.listdir(directory):
        if filename.endswith(".g6"):
            file_path = os.path.join(directory, filename)
            print(f"Parsing {filename}")
            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if line:
                        try:
                            graph = nx.from_graph6_bytes(line.encode())
                            graphs.append(graph)
                        except Exception as e:
                            print(f"Error parsing {filename}: {e}")    
    return graphs


def spectral_radius_lm(G):
    real_value = max(abs(np.linalg.eigvals(nx.laplacian_matrix(G).toarray())))
    return real_value

def spectral_radius_am(G):
    real_value = max(abs(np.linalg.eigvals(nx.adjacency_matrix(G).toarray())))
    return real_value

funcs["spectral_radius_adjacency"] = spectral_radius_am
funcs["spectral_radius_laplacian"] = spectral_radius_lm

t = time.time()
nx_list, _, _, _, _ = restore_checkpoint(checkpoint_dir="../data/checkpoints")
t = reset_time_with_message(message="restored checkpoints", t=t)
databse_graphs = parse_g6_directory("../../data/database")
t = reset_time_with_message(message="parsed complete database (2 - 8)", t=t)
nx_list.extend(databse_graphs)
t = reset_time_with_message(message="lists merged", t=t)

graphs_lists = {n_nodes: [] for n_nodes in range(2, 51)}
for i, G in enumerate(nx_list):
    invariants = calc_invariants(G)
    invariant_values = [invariants[key] for key in funcs.keys()]
    graph_data = {
        "graph_features": invariant_values,
        "nodes": list(G.nodes()),
        "edges": list(G.edges())
    }
    graphs_lists[invariants['nodes_count']].append(graph_data)
    if (i + 1) % 10000 == 0:
        t = reset_time_with_message(message=f"processed {i + 1} graphs", t=t)

print("dictionary created")

for n_nodes in graphs_lists.keys():
    data = {
        'nodes_count': n_nodes,
        'invariants_order': list(funcs.keys()),
        'graph_list': graphs_lists[n_nodes]
    }
    with open(f"../data/dataset/graphs{n_nodes}.json", "w") as f:
        json.dump(data, f)
    t = reset_time_with_message(message=f"../data/dataset/graphs{n_nodes}.json written", t=t)


restored checkpoints : 47.25639796257019 s
Parsing graph5.g6
Parsing graph4.g6
Parsing graph3.g6
Parsing graph7.g6
Parsing graph6.g6
Parsing graph2.g6
Parsing graph8.g6
parsed complete database (2 - 8) : 23.563014268875122 s
lists merged : 0.0030128955841064453 s


<class 'networkx.utils.decorators.argmap'> compilation 9:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/var/folders/xr/j1k993bs101gs699fxggjf3h0000gn/T/ipykernel_50789/3304937154.py:36: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  real_value = max(abs(np.linalg.eigvals(nx.adjacency_matrix(G).toarray())))


processed 10000 graphs : 37.53977966308594 s
processed 20000 graphs : 36.37114763259888 s
processed 30000 graphs : 34.88269329071045 s
processed 40000 graphs : 35.70436930656433 s
processed 50000 graphs : 38.33825206756592 s
processed 60000 graphs : 35.98636293411255 s
processed 70000 graphs : 66.27533721923828 s
processed 80000 graphs : 41.00601387023926 s
processed 90000 graphs : 32.133325815200806 s
processed 100000 graphs : 35.81530809402466 s
processed 110000 graphs : 31.577635049819946 s
processed 120000 graphs : 37.96307897567749 s
processed 130000 graphs : 37.55972719192505 s
processed 140000 graphs : 65.37597918510437 s
processed 150000 graphs : 35.36451005935669 s
processed 160000 graphs : 35.31768012046814 s
processed 170000 graphs : 39.59867477416992 s
processed 180000 graphs : 35.51334190368652 s
processed 190000 graphs : 40.21569299697876 s
processed 200000 graphs : 34.30967903137207 s
processed 210000 graphs : 33.66223096847534 s
processed 220000 graphs : 65.196143865585

In [ ]:
import os
import json

def read_parse_json_files(directory_path):
    graphs = []
    i = 0
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            print(f"{i + 1} reading {file_path}")
            i += 1
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    graphs.extend(data['graph_list'])
            except json.JSONDecodeError as e:
                print(f"Error parsing {filename}: {e}")
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    
    return graphs

graphs = read_parse_json_files('../data/dataset')
print(len(graphs))